In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.15.0


In [3]:
MAX_LEN = 300
BATCH_SIZE = 32
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.reuters.load_data()

2110848/2110848 [==============================] - 0s 0us/step


In [8]:
print(x_train.shape)

(8982, 300)


In [4]:
x_train = tf.keras.utils.pad_sequences(x_train, maxlen=MAX_LEN, padding='post')
x_test = tf.keras.utils.pad_sequences(x_test, maxlen=MAX_LEN, padding='post')

MAX_WORDS = x_train.max()+1
CAT_NUM = y_train.max()+1

ds_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)) \
          .shuffle(buffer_size = 1000).batch(BATCH_SIZE) \
          .prefetch(tf.data.experimental.AUTOTUNE).cache()

ds_test = tf.data.Dataset.from_tensor_slices((x_test,y_test)) \
          .shuffle(buffer_size = 1000).batch(BATCH_SIZE) \
          .prefetch(tf.data.experimental.AUTOTUNE).cache()

In [9]:
def create_model():

    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(300,)))
    model.add(tf.keras.layers.Embedding(MAX_WORDS,7))
    model.add(tf.keras.layers.Conv1D(filters = 64,kernel_size = 5,activation = "relu"))
    model.add(tf.keras.layers.MaxPool1D(2))
    model.add(tf.keras.layers.Conv1D(filters = 32,kernel_size = 3,activation = "relu"))
    model.add(tf.keras.layers.MaxPool1D(2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(CAT_NUM, activation = "softmax"))
    return(model)

def compile_model(model):
    model.compile(optimizer=tf.keras.optimizers.Nadam(),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(5)])
    return(model)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 7)            216874    
                                                                 
 conv1d_4 (Conv1D)           (None, 296, 64)           2304      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 148, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_5 (Conv1D)           (None, 146, 32)           6176      
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 73, 32)            0         
 g1D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2336)             

In [10]:
#增加以下两行代码
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = create_model()
    model.summary()
    model = compile_model(model)

history = model.fit(ds_train,validation_data = ds_test,epochs = 10)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 7)            216874    
                                                                 
 conv1d_6 (Conv1D)           (None, 296, 64)           2304      
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 148, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_7 (Conv1D)           (None, 146, 32)           6176      
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, 73, 32)            0         
 g1D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 2336)             